In [29]:
""" Utilities """

def Union(lst1, lst2): 
    return list(set(lst1) | set(lst2)) 

def Complement(list1, list2):
    return list(set(list1) - set(list2))

In [30]:
""" Import data from csv file """

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import data from csv file
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# display data with plot
# df_with_nan = df[df.isna().any(axis=1)]
# df_with_nan.head(10)
# drop data with too many NaN

train_df = df.sample(frac=0.8, random_state=99)
validate_df = df.drop(train_df.index)

train_df_original = train_df.copy()
validate_df_original = validate_df.copy()
test_df_original = test_df.copy()

In [31]:
""" Preprocessing data """
# Dealing the Discrete data
# Translating discrete data into number

discrete_tags = ["MSZoning", "Street", "Alley", "LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope",
                 "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl",
                 "Exterior1st", "Exterior2nd", "MasVnrType", "ExterQual", "ExterCond", "Foundation", "BsmtQual",
                 "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Heating", "HeatingQC", "CentralAir",
                 "Electrical", "KitchenQual", "Functional", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual",
                 "GarageCond", "PavedDrive", "PoolQC", "Fence", "MiscFeature", "SaleType", "SaleCondition"]

# divide data into discrete and continuous
train_df_continuous = train_df.drop(columns=discrete_tags)
train_df_discrete = train_df[discrete_tags]
validate_df_continuous = validate_df.drop(columns=discrete_tags)
validate_df_discrete = validate_df[discrete_tags]
test_df_continuous = test_df.drop(columns=discrete_tags)
test_df_discrete = test_df[discrete_tags]

train_df_discrete = pd.get_dummies(train_df_discrete, columns=discrete_tags, dtype='int')
validate_df_discrete = pd.get_dummies(validate_df_discrete, columns=discrete_tags, dtype='int')
test_df_discrete = pd.get_dummies(test_df_discrete, columns=discrete_tags, dtype='int')

# dealing with discrete data with missing columns
columns = Union(train_df_discrete.columns.tolist(), validate_df_discrete.columns.tolist())
columns = Union(columns, test_df_discrete.columns.tolist())

for tags_missing in Complement(columns, train_df_discrete.columns.tolist()):
    train_df_discrete[tags_missing] = 0

for tags_missing in Complement(columns, validate_df_discrete.columns.tolist()):
    validate_df_discrete[tags_missing] = 0

for tags_missing in Complement(columns, test_df_discrete.columns.tolist()):
    test_df_discrete[tags_missing] = 0

train_df = train_df_continuous.join(train_df_discrete)
validate_df = validate_df_continuous.join(validate_df_discrete)
test_df = test_df_continuous.join(test_df_discrete)

train_df_discrete['SalePrice'] = train_df_original['SalePrice']

print("discrete columns: ", columns)

discrete columns:  ['LotConfig_CulDSac', 'Exterior2nd_BrkFace', 'HouseStyle_2.5Unf', 'GarageType_2Types', 'RoofMatl_WdShngl', 'Fence_MnWw', 'RoofStyle_Flat', 'GarageType_Detchd', 'ExterQual_Ex', 'ExterCond_Gd', 'BsmtQual_Gd', 'Condition1_PosN', 'LotConfig_FR3', 'GarageCond_Gd', 'Exterior2nd_AsphShn', 'GarageCond_Po', 'GarageFinish_Fin', 'SaleType_ConLI', 'HouseStyle_SLvl', 'Functional_Mod', 'Condition1_Feedr', 'MiscFeature_Shed', 'GarageQual_Po', 'Exterior1st_VinylSd', 'ExterQual_Gd', 'BsmtFinType2_Rec', 'GarageQual_Fa', 'CentralAir_Y', 'SaleType_Con', 'Exterior2nd_ImStucc', 'BsmtFinType2_BLQ', 'LandContour_Low', 'MiscFeature_Gar2', 'HouseStyle_1Story', 'SaleCondition_AdjLand', 'MSZoning_RL', 'Neighborhood_Sawyer', 'Exterior1st_Stone', 'Exterior2nd_Wd Sdng', 'Neighborhood_Mitchel', 'Neighborhood_SWISU', 'MasVnrType_BrkCmn', 'FireplaceQu_Ex', 'FireplaceQu_TA', 'Exterior2nd_MetalSd', 'SaleCondition_Alloca', 'Condition1_RRAn', 'Foundation_CBlock', 'GarageQual_Ex', 'Functional_Maj1', 'Gara

In [32]:
""" Process & Train data """
# getting correlation between each column

from sklearn.preprocessing import StandardScaler
train_df_corr_continuous = train_df_continuous.corr(numeric_only=True)
train_df_corr_discrete = train_df_discrete.corr(numeric_only=True)
corr_result_continuous = train_df_corr_continuous['SalePrice']
corr_result_discrete = train_df_corr_discrete['SalePrice']

tags_to_use = []
tags_to_drop = []

# use correlation to select continuous tags
print("corr_result_continuous: ", corr_result_continuous)
for i in range(corr_result_continuous.size):
    if ((abs(corr_result_continuous[i]) >= 0.7)):
        if corr_result_continuous.index[i] != "SalePrice":
            tags_to_use.append(corr_result_continuous.index[i])

# use correlation to select discrete tagsD
print("corr_result_discrete: ", corr_result_discrete)
for i in range(corr_result_discrete.size):
    if ((abs(corr_result_discrete[i]) >= 0.7)):
        if corr_result_discrete.index[i] != "SalePrice":
            tags_to_use.append(corr_result_discrete.index[i])

for i in train_df.columns:
    if i == "SalePrice":
        continue

    if i not in tags_to_use:
        tags_to_drop.append(i)
    else:
        # Replace NaN with mean value
        train_df[i] = train_df[i].fillna(train_df[i].mean())
        validate_df[i] = test_df[i].fillna(test_df[i].mean())
        test_df[i] = test_df[i].fillna(test_df[i].mean())

print('tags used: ', tags_to_use)

# train data
tags_to_drop_with_saleprice = tags_to_drop.copy() + ["SalePrice"]

y_train_df = train_df["SalePrice"]
train_df = train_df.drop(columns=(tags_to_drop_with_saleprice))
y_validate_df = validate_df["SalePrice"]
validate_df = validate_df.drop(columns=(tags_to_drop_with_saleprice))

# test data
# SalePrice is added owing to adding missing columns
test_df = test_df.drop(columns=tags_to_drop)

# print("train_df: ", train_df.columns.tolist())
# print("validate_df: ", validate_df.columns.tolist())
# print("test_df: ", test_df.columns.tolist())

# Standardize data

scaler = StandardScaler()
train_df = scaler.fit_transform(train_df)
validate_df = scaler.transform(validate_df)
test_df = scaler.transform(test_df)

corr_result_continuous:  Id              -0.045039
MSSubClass      -0.084145
LotFrontage      0.367859
LotArea          0.259909
OverallQual      0.788594
OverallCond     -0.074155
YearBuilt        0.513863
YearRemodAdd     0.507041
MasVnrArea       0.498025
BsmtFinSF1       0.377481
BsmtFinSF2      -0.008659
BsmtUnfSF        0.217491
TotalBsmtSF      0.608674
1stFlrSF         0.608352
2ndFlrSF         0.322648
LowQualFinSF    -0.005624
GrLivArea        0.711333
BsmtFullBath     0.215781
BsmtHalfBath    -0.008694
FullBath         0.560334
HalfBath         0.279793
BedroomAbvGr     0.170780
KitchenAbvGr    -0.132752
TotRmsAbvGrd     0.546397
Fireplaces       0.475380
GarageYrBlt      0.487397
GarageCars       0.647004
GarageArea       0.633463
WoodDeckSF       0.333595
OpenPorchSF      0.307736
EnclosedPorch   -0.126583
3SsnPorch        0.045775
ScreenPorch      0.122526
PoolArea         0.029224
MiscVal         -0.011121
MoSold           0.040157
YrSold          -0.027366
SalePrice    

In [33]:
""" Train model """
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, SGDRegressor
from tqdm import tqdm
import itertools

for x, y in tqdm(itertools.product(range(0.1, 0.7, 0.1), range(0.1, 0.7, 0.1))):
    
    pass


# Linear Regression
regressor = LinearRegression()
regressor.fit(train_df, y_train_df)
validate_df_predict_lr = regressor.predict(validate_df)

# SGD Regressor
sgd_regressor = SGDRegressor()
sgd_regressor.fit(train_df, y_train_df)
validate_df_predict_sgd = sgd_regressor.predict(validate_df)

# Polynomial Regression
polynomial_degree = 3

poly_reg = PolynomialFeatures(degree=polynomial_degree)
train_df_poly = poly_reg.fit_transform(train_df)
poly_regressor = LinearRegression().fit(train_df_poly, y_train_df)

validate_df_poly = PolynomialFeatures(
    degree=polynomial_degree).fit_transform(validate_df)
polynomial_predict = poly_regressor.predict(validate_df_poly)

""" Evaluate models """
lr_mean_square = mean_squared_error(y_validate_df, validate_df_predict_lr)
sgd_mean_square = mean_squared_error(y_validate_df, validate_df_predict_sgd)
print("root mean square(LR): ", '{:,}'.format(lr_mean_square))
print("root mean square(SGD): ", '{:,}'.format(sgd_mean_square))
print("root mean square(Poly): ", '{:,}'.format(
    mean_squared_error(y_validate_df, polynomial_predict)))

root mean square(LR):  9,727,478,669.02359
root mean square(SGD):  9,580,561,601.927967
root mean square(Poly):  10,150,753,896.92923


In [34]:
""" Predict """
## Linear Regression
test_df_predict = regressor.predict(test_df)
# save the result to csv file
dataframe_result = pd.DataFrame({'Id': test_df_original.Id, 'SalePrice': test_df_predict})
dataframe_result.to_csv('result_lr.csv', index=False)

## SGD Regressor
test_df_predict = sgd_regressor.predict(test_df)
# save the result to csv file
dataframe_result = pd.DataFrame({'Id': test_df_original.Id, 'SalePrice': test_df_predict})
dataframe_result.to_csv('result_sgd.csv', index=False)

In [35]:
""" Visualize """
import matplotlib.pyplot as plt
# visualize the training set results
plt.scatter(train_df["1stFlrSF"], y_train_df, color = 'red')
plt.plot(train_df["1stFlrSF"], regressor.predict(train_df), color = 'blue')
plt.title('Environment vs SalesPrice (trainning set)')
plt.xlabel("Environment")
plt.ylabel("Sales Price")
plt.show()

# visualize the test set results
plt.scatter(test_df["1stFlrSF"], y_test_df, color = 'red')
plt.plot(test_df["1stFlrSF"], regressor.predict(test_df), color = 'blue')
plt.title('Environment vs SalesPrice (test set)')
plt.xlabel("Environment")
plt.ylabel("Sales Price")
plt.show()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices